### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩 :   
    *__int__() : 모델 층 구성, 설계  
    *forward() : 순방향 학습 진행 코드 구현  
-동적 모델

In [21]:
## 모듈로딩
import torch    # 텐서 관련 모듈
import torch.nn as nn # 인공신경망 관련 모듈
import torch.nn.functional as F # 인공신경망 관련 함수들 모듈 ( 손실 함수, 활성화함수 등등 )
import torch.optim as optim # 최적화 관련 모듈 ( 가중치 , 절편 빠르게 찾아주는 알고리즘 )
from torchinfo import summary # 모델 구조 및 저보 관련 모듈
from torchmetrics.regression import * # 회귀 성능 관련
from torchmetrics.classification import * # 분류 성능 관련

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [22]:
# 랜덤 고정
torch.manual_seed(1)
# 텐서 저장 및 실행 위치설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

### 모델 설계 ( 동적 모델 )
- 목표 : 은닉층의 개수가 동적인 모델
- 조건
    * 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값이 필요함   
    * 은닉층의 개수 동적 + 퍼셉트론 개수 고정 => 은닉층의 개수, 퍼셉트론 수  

In [23]:
#  모델 이름 : DynamicModel
#  모델의 매개변수 : in_in, out_out, h_inout, h_cnt
#  동적 모델 
#   container 모듈 중. nn.ModuleList() 사용해서 동적으로 Layer 추가
#       forwoard 기능 미 제공
#       layer 인스턴스 요소 사이에 연관성 없음
#       layer 인스턴스 요소는 인덱싱으로 접근
class DynamicModel(nn.Module):
    def __init__(self,in_in, out_out, h_inout, h_cnt):
        super().__init__()

        self.in_layer = nn.Linear(in_in, h_inout)
        self.h_layers =nn.ModuleList( [ nn.Linear(h_inout, h_inout) for _ in range(h_cnt)] )
        self.out_layer = nn.Linear(h_inout, out_out)

    #학습 진행 콜백 메서드
    def forward(self,x):
        # 입력층
        y=self.in_layer(x)   # y=x1w1+x2w2+x3w3 + b
        y=F.relu(y)          # 0 <= y
        #은니층
        for linear in self.h_layers:
            y=linear(y)
            y=F.relu(y)
        
        # 출력층
        return self.out_layer(y)
        
    
# 모델인스턴스 생성
m1 = DynamicModel(3, 2, 5, 10)

# 모델 파라미터 확인
for name, param in m1.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
h_layers.0.weight torch.Size([5, 5])
h_layers.0.bias torch.Size([5])
h_layers.1.weight torch.Size([5, 5])
h_layers.1.bias torch.Size([5])
h_layers.2.weight torch.Size([5, 5])
h_layers.2.bias torch.Size([5])
h_layers.3.weight torch.Size([5, 5])
h_layers.3.bias torch.Size([5])
h_layers.4.weight torch.Size([5, 5])
h_layers.4.bias torch.Size([5])
h_layers.5.weight torch.Size([5, 5])
h_layers.5.bias torch.Size([5])
h_layers.6.weight torch.Size([5, 5])
h_layers.6.bias torch.Size([5])
h_layers.7.weight torch.Size([5, 5])
h_layers.7.bias torch.Size([5])
h_layers.8.weight torch.Size([5, 5])
h_layers.8.bias torch.Size([5])
h_layers.9.weight torch.Size([5, 5])
h_layers.9.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


- 학습 진행

In [26]:
## 임시 데이터 생성
datats = torch.FloatTensor([[1,3,5], [2,4,6], [3,5,7], [4,6,8]]) # 4행 3열
targetts = torch.FloatTensor([[10,9], [8,2], [3,9], [10,12]]) # 4행 2열

# 모델 학습
d=m1(datats)
print(d, d.shape)

tensor([[0.1237, 0.0215],
        [0.1237, 0.0215],
        [0.1237, 0.0215],
        [0.1237, 0.0215]], grad_fn=<AddmmBackward0>) torch.Size([4, 2])


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyDynamicModel3(nn.Module):
    def __init__(self, in_feature, hidden_layers, out_feature=1):
        super(MyDynamicModel3, self).__init__()

        # 1. 입력층 정의
        self.input_layer = nn.Linear(in_feature, hidden_layers[0] if hidden_layers else out_feature)
        
        # 2. 은닉층을 동적으로 정의 (빈 리스트가 들어오면 은닉층을 생략)
        self.hidden_layers = nn.ModuleList()

        for i in range(1, len(hidden_layers)):
            self.hidden_layers.append(nn.Linear(hidden_layers[i - 1], hidden_layers[i]))

        # 3. 출력층 정의 (은닉층이 없으면 입력층에서 바로 출력층으로 연결)
        self.output_layer = nn.Linear(hidden_layers[-1] if hidden_layers else in_feature, out_feature)

    def forward(self, x):
        # 입력층 처리
        x = F.relu(self.input_layer(x))

        # 은닉층 처리 (은닉층이 있다면 처리)
        for layer in self.hidden_layers:
            x = F.relu(layer(x))

        # 출력층 처리
        return self.output_layer(x)

# 은닉층 없이 모델을 구성 (입력층 -> 출력층만 존재)
model = MyDynamicModel3(in_feature=10, hidden_layers=[10,20,30,40,50,60], out_feature=3)
print(model)

MyDynamicModel3(
  (input_layer): Linear(in_features=10, out_features=10, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=30, bias=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): Linear(in_features=40, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=60, bias=True)
  )
  (output_layer): Linear(in_features=60, out_features=3, bias=True)
)
